In [1]:
import itertools
import random
import numpy as np
import gym
import time
from gym import spaces
from typing import Tuple, List
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from envs.broken_components import BrokenComponentsEnv
from envs.data_handler import DataHandler
from IPython.display import clear_output

In [2]:
dh = DataHandler()
broken_components = dh.get_sample_component_failure_pairs(5)
broken_components

[('Last Second Sales Item Filter', 'CF3'),
 ('Availability Item Filter', 'CF1'),
 ('Future Sales Item Filter', 'CF1'),
 ('Reputation Service', 'CF1'),
 ('Past Sales Item Filter', 'CF3')]

## Environment

In [3]:
# possible reward_modus 'raw', 'sqt', 'log10', 'cube'
env = BrokenComponentsEnv(broken_components, reward_modus='raw')

### Env properties

In [4]:
n_actions = env.action_space.n
n_actions

5

In [5]:
n_spaces = env.observation_space.n
n_spaces

32

In [6]:
env.action_space_names

array([('Last Second Sales Item Filter', 'CF3'),
       ('Availability Item Filter', 'CF1'),
       ('Future Sales Item Filter', 'CF1'), ('Reputation Service', 'CF1'),
       ('Past Sales Item Filter', 'CF3')], dtype=object)

In [7]:
env.observation_space_names

[array([('Last Second Sales Item Filter', 'CF3'),
        ('Availability Item Filter', 'CF1'),
        ('Future Sales Item Filter', 'CF1'), ('Reputation Service', 'CF1'),
        ('Past Sales Item Filter', 'CF3')], dtype=object),
 array([('Last Second Sales Item Filter', 'CF3'),
        ('Availability Item Filter', 'CF1'),
        ('Future Sales Item Filter', 'CF1'), ('Reputation Service', 'CF1')],
       dtype=object),
 array([('Last Second Sales Item Filter', 'CF3'),
        ('Availability Item Filter', 'CF1'),
        ('Future Sales Item Filter', 'CF1'),
        ('Past Sales Item Filter', 'CF3')], dtype=object),
 array([('Last Second Sales Item Filter', 'CF3'),
        ('Availability Item Filter', 'CF1'),
        ('Future Sales Item Filter', 'CF1')], dtype=object),
 array([('Last Second Sales Item Filter', 'CF3'),
        ('Availability Item Filter', 'CF1'), ('Reputation Service', 'CF1'),
        ('Past Sales Item Filter', 'CF3')], dtype=object),
 array([('Last Second Sales Item Fil

### Examples

#### Example 1

In [8]:
env.reset(reward_modus='raw') # possible reward_modus 'raw', 'sqt', 'log10', 'cubic'
env.render()

action = env.action_space.sample()
state, reward, done, _ = env.step(action)
print(state, reward, done)
env.render()

action = env.action_space.sample()
state, reward, done, _ = env.step(action)
print(state, reward, done)
env.render()

action = env.action_space.sample()
state, reward, done, _ = env.step(action)
print(state, reward, done)
env.render()

Steps:  0
Action:  None
Successful:  None
State:  [('Last Second Sales Item Filter', 'CF3'), ('Availability Item Filter', 'CF1'), ('Future Sales Item Filter', 'CF1'), ('Reputation Service', 'CF1'), ('Past Sales Item Filter', 'CF3')] 


2 16.0 False
Steps:  1
Action:  ('Reputation Service', 'CF1')
Successful:  True
State:  [('Last Second Sales Item Filter', 'CF3'), ('Availability Item Filter', 'CF1'), ('Future Sales Item Filter', 'CF1'), ('Past Sales Item Filter', 'CF3')] 


3 145.4401264 False
Steps:  2
Action:  ('Past Sales Item Filter', 'CF3')
Successful:  True
State:  [('Last Second Sales Item Filter', 'CF3'), ('Availability Item Filter', 'CF1'), ('Future Sales Item Filter', 'CF1')] 


3 0 False
Steps:  3
Action:  ('Future Sales Item Filter', 'CF1')
Successful:  False
State:  [('Last Second Sales Item Filter', 'CF3'), ('Availability Item Filter', 'CF1'), ('Future Sales Item Filter', 'CF1')] 




#### Example 2

In [9]:
env.reset('raw') # possible reward_modus 'raw', 'sqt', 'log10', 'cube'
accumulated_reward = 0
episode_done = False

while(not episode_done):
    action = env.action_space.sample()
    state, reward, done, _ = env.step(action)
    accumulated_reward += reward
    
    clear_output(wait=True)
    env.render()
    time.sleep(1)
    
    
    if(done):
        print('FINISHED!')
        print('Acc Reward: ', accumulated_reward)
    episode_done = done

Steps:  5
Action:  ('Reputation Service', 'CF1')
Successful:  False
State:  [('Availability Item Filter', 'CF1'), ('Future Sales Item Filter', 'CF1')] 




KeyboardInterrupt: 